In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import math
import os
import datetime
from pandas import DataFrame
import itertools
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
import torch
import xmltodict
import xml.etree.ElementTree as ET
import xml
from xml.etree.ElementTree import parse
from xml.parsers import expat
from xml.dom.minidom import parse, parseString
import psycopg2
import PyPDF2
import pikepdf
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
username='joythompson'
directorymidtransform='/Users/'+username+'/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/transitional_transform_data/'
directoryraw='/Users/'+username+'/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/Raw_Data_Files/'
dbname='postgres'
conn = psycopg2.connect(database=dbname, user=username, password='', host='127.0.0.1', port='5432')
conn.autocommit = True
cursor = conn.cursor()
queryschema='Create schema dse203politics'

OperationalError: connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


***REPRESENTATIVES***

In [3]:
#cursor.execute(queryschema)
repinfo=pd.read_csv(directoryraw+ 'us-115th-congress-members-fixed.csv')
queryrep='CREATE TABLE dse203politics.StateReps (RepresentativeID SERIAL PRIMARY KEY NOT NULL, LastName varchar(255), \
    FirstName varchar(255), StateLabel varchar(255), StateCode int, District int, Chamber varchar(255), Party varchar(255))'
cursor.execute(queryrep)
countind=0
for ind in repinfo.index:
    first=repinfo.loc[ind, 'First Name'][1:].replace("'", '').replace('.','')
    if len(first.split(' ')[0])==1:
        first=first.split(' ')[1]+' '+first.split(' ')[0]
    queryinsert="INSERT INTO dse203politics.StateReps  VALUES ("+str(countind)+", "+str("'"+repinfo.loc[ind, 'Last Name'].replace("'", '')+"'")+ ", "+ \
        str("'"+first+"'")+ ", "+ str("'"+repinfo.loc[ind, 'State Label']+"'")+ ", "+str(repinfo.loc[ind, 'State Code'])+ ", "+str(repinfo.loc[ind, 'District'])+ ", " \
        +str("'"+repinfo.loc[ind, 'Chamber']+"'") +", "+str("'"+repinfo.loc[ind, 'Party']+"'")+ ")" 
    cursor.execute(queryinsert)
    countind=countind+1

FileNotFoundError: [Errno 2] No such file or directory: '/Users/joythompson/DSE_203_JoyT/DSE_203_Final_Project_JoyT/Raw_Data_Files/us-115th-congress-members-fixed.csv'

***COMMIITTEES***

In [28]:
reppd=pd.read_csv(directorymidtransform +'/unique_legislator_committee_relationship_by_line.csv').drop(columns=["Unnamed: 0"])
commpd=reppd.groupby('Committee').sum().reset_index().reset_index()
commpd=commpd.rename(columns={'index':'CommitteeID'}).drop(0).set_index('Committee')
droplist=[]
for ind in reppd.index:
    if reppd.loc[ind, 'Committee']!='Abraham':
        reppd.loc[ind, 'CommitteeID']=commpd.loc[reppd.loc[ind, 'Committee'], 'CommitteeID']
    else:
        droplist.append(ind)
reppd=reppd.drop(droplist)

In [ ]:
querycomm='CREATE TABLE dse203finalproject.Committees (RepCommID SERIAL PRIMARY KEY NOT NULL, RepresentativeName varchar(255), \
    FirstName varchar(255), State varchar(255), CommitteeName varchar(255), CommitteeID varchar(255), CommitteeRole varchar(255))'
#cursor.execute(querycomm)
countind=0
for ind in reppd.index:
    queryinsert="INSERT INTO dse203finalproject.Committees  VALUES ("+str(countind)+", "+str("'"+reppd.loc[ind, 'Representative Name'].replace("'", '')+"'")+ \
        ", "+ str("'"+reppd.loc[ind, 'Last Name'].replace("'", '')+"'")+ ", " +str("'"+reppd.loc[ind, 'State']+"'")+ ", "+ \
        str("'"+reppd.loc[ind, 'Committee'].replace("'", '')+"'")+", "+ str("'"+reppd.loc[ind, 'CommiteeID'].replace("'", '')+"'") +", "+ str("'"+reppd.loc[ind, 'Role']+"'")+  ")" 
    #cursor.execute(queryinsert)
    countind=countind+1

***COMMIITTEES Referencing REPRESENTATIVE ID***

In [ ]:
querycomm2='CREATE TABLE dse203finalproject.CommitteesbyID(SeatID SERIAL PRIMARY KEY NOT NULL, RepresentativeID INT references \
    dse203finalproject.StateReps (RepresentativeID) NOT NULL, CommitteeName varchar(255), Chair varchar(255))'    
#cursor.execute(querycomm2)
querycomm2in='INSERT INTO dse203finalproject.CommitteesbyID (RepresentativeID, CommitteeName, Chair) \
Select StateReps.RepresentativeID as "RepresentativeID", Committees.CommitteeName as "CommitteeName", \
    Committees.CommitteeRole as "CommitteeRole" from dse203finalproject.Committees \
    left join dse203finalproject.StateReps on Committees.RepresentativeName = StateReps.LastName \
    where Committees.State is NULL and StateReps.RepresentativeID is NOT NULL \
    UNION \
    Select StateReps.RepresentativeID as "RepresentativeID", Committees.CommitteeName as "CommitteeName", \
    Committees.CommitteeRole as "CommitteeRole" from dse203finalproject.Committees \
    left join dse203finalproject.StateReps on Committees.RepresentativeName = StateReps.LastName \
    and Committees.State = StateReps.Statelabel where Committees.State is NOT NULL and \
    StateReps.RepresentativeID is NOT NULL'
#cursor.execute(querycomm2in)

***BILLS***

In [ ]:
queryxml='CREATE TABLE dse203finalproject.xmlbaseraw (BillID SERIAL PRIMARY KEY NOT NULL, Link varchar(255), OfficialTitle varchar(2500), \
            BillStage varchar(255), LegisType varchar(255))'
#cursor.execute(queryxml)
queryxmldate='CREATE TABLE dse203finalproject.xmlbaserawdate (BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, Date varchar(255))'
#cursor.execute(queryxmldate)
queryxmlcomm='CREATE TABLE dse203finalproject.xmlbaserawcommittee (BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, CommitteeName varchar(255))'
#cursor.execute(queryxmlcomm)
queryxmlspon='CREATE TABLE dse203finalproject.xmlbaserawsponsor (BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, FirstName varchar(255), LastName varchar(255), State varchar(255))'
#cursor.execute(queryxmlspon)
queryxmlcospon='CREATE TABLE dse203finalproject.xmlbaserawcosponsor (BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, FirstName varchar(255), LastName varchar(255), State varchar(255))'
#cursor.execute(queryxmlcospon)
countind=0
for ind in xmlnames.index:
    link= str("'"+xmlnames.loc[ind, 'Link'].replace("'", '')+"'")
    title=str("'"+xmlnames.loc[ind, 'OfficialTitle'].replace("'", '')+"'")
    stage=str("'"+xmlnames.loc[ind, 'BillStage']+"'")
    legis=str("'"+xmlnames.loc[ind, 'LegisType']+"'")
    queryxinsert="INSERT INTO dse203finalproject.xmlbaseraw   VALUES ("+str(countind)+", "+link+ \
        ", "+title + ", " +stage+ ", " +legis+")" 
    #cursor.execute(queryxinsert)
    if '[' not in str(xmlnames.loc[ind, 'Date']) and str(xmlnames.loc[ind, 'Date'])!='':
        date=str("'"+xmlnames.loc[ind, 'Date'].replace("'", '')+"'")
        queryxinsertd="INSERT INTO dse203finalproject.xmlbaserawdate   VALUES ("+str(countind)+", "+date+")"
        #cursor.execute(queryxinsertd)
    elif str(xmlnames.loc[ind, 'Date'])!='':
        dates=xmlnames.loc[ind, 'Date']
        for dateind in dates:
            date=str("'"+dateind.replace("'", '')+"'")
            queryxinsertd="INSERT INTO dse203finalproject.xmlbaserawdate  VALUES ("+str(countind)+", "+date+")"
            #cursor.execute(queryxinsertd)
    if '[' not in str(xmlnames.loc[ind, 'CommitteeName']) and str(xmlnames.loc[ind, 'CommitteeName'])!='':
        comm=str("'"+xmlnames.loc[ind, 'CommitteeName'].replace("'", '')+"'")
        queryxinsertc="INSERT INTO dse203finalproject.xmlbaserawcommittee   VALUES ("+str(countind)+", "+comm+")"
        #cursor.execute(queryxinsertc)
    elif str(xmlnames.loc[ind, 'CommitteeName'])!='':
        comms=xmlnames.loc[ind, 'CommitteeName']
        for commind in comms:
            comm=str("'"+commind.replace("'", '')+"'")
            queryxinsertc="INSERT INTO dse203finalproject.xmlbaserawcommittee  VALUES ("+str(countind)+", "+comm+")"
            #cursor.execute(queryxinsertc)
    if '[' not in str(xmlnames.loc[ind, 'SponsorLastName']) and str(xmlnames.loc[ind, 'SponsorLastName'])!='':
        queryxinsertssp="INSERT INTO dse203finalproject.xmlbaserawsponsor  VALUES ("+str(countind)+", "+str("'"+xmlnames.loc[ind, 'SponsorFirstName'].replace("'", '')+"'") \
        + ", "+str("'"+xmlnames.loc[ind, 'SponsorLastName'].replace("'", '')+"'")+ ", "+str("'"+xmlnames.loc[ind, 'SponsorState']+"'")+")" 
        #cursor.execute(queryxinsertssp)
    elif str(xmlnames.loc[ind, 'SponsorLastName'])!='':
        lasts=xmlnames.loc[ind, 'SponsorLastName']
        countnames=0
        for lastind in lasts:
            last=str("'"+lastind.replace("'", '')+"'")
            first=str("'"+xmlnames.loc[ind, 'SponsorFirstName'][countnames].replace("'", '')+"'")
            state=str("'"+xmlnames.loc[ind, 'SponsorState'][countnames]+"'")
            queryxinsertssp="INSERT INTO dse203finalproject.xmlbaserawsponsor  VALUES ("+str(countind)+", "+first+", "+last+ ", "+state+")"
            #cursor.execute(queryxinsertssp)
            countnames=countnames+1
    if '[' not in str(xmlnames.loc[ind, 'CoSponsorLastName']) and str(xmlnames.loc[ind, 'CoSponsorLastName'])!='':
        queryxinsertsco="INSERT INTO dse203finalproject.xmlbaserawcosponsor  VALUES ("+str(countind)+", "+str("'"+xmlnames.loc[ind, 'CoSponsorFirstName'].replace("'", '')+"'")+ \
        ", "+str("'"+xmlnames.loc[ind, 'CoSponsorLastName'].replace("'", '')+"'")+ ", "+str("'"+xmlnames.loc[ind, 'CoSponsorState']+"'")+")" 
        #cursor.execute(queryxinsertsco)
    elif str(xmlnames.loc[ind, 'CoSponsorLastName'])!='':
        lasts=xmlnames.loc[ind, 'CoSponsorLastName']
        countnames=0
        for lastind in lasts:
            last=str("'"+lastind.replace("'", '')+"'")
            first=str("'"+xmlnames.loc[ind, 'CoSponsorFirstName'][countnames].replace("'", '')+"'")
            state=str("'"+xmlnames.loc[ind, 'CoSponsorState'][countnames]+"'")
            queryxinsertsco="INSERT INTO dse203finalproject.xmlbaserawcosponsor  VALUES ("+str(countind)+", "+first+", "+last+ ", "+state+")"
            #cursor.execute(queryxinsertsco)
            countnames=countnames+1
        
    countind=countind+1

***BILLS Referencing REPRESENTATIVE ID***

In [ ]:
queryspon2='CREATE TABLE dse203finalproject.SponsorsbyID(BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, RepresentativeID INT references \
    dse203finalproject.StateReps (RepresentativeID) NOT NULL)'    
#cursor.execute(queryspon2)
querysponin2='INSERT INTO dse203finalproject.SponsorsbyID (BillID, RepresentativeID) \
Select xmlbaserawsponsor.BillID as "BillID", StateReps.RepresentativeID as "RepresentativeID" from dse203finalproject.xmlbaserawsponsor \
    left join dse203finalproject.StateReps on xmlbaserawsponsor.LastName = StateReps.LastName \
    where xmlbaserawsponsor.State is NULL and StateReps.RepresentativeID is NOT NULL \
    UNION \
    Select xmlbaserawsponsor.BillID as "BillID", StateReps.RepresentativeID as "RepresentativeID" from dse203finalproject.xmlbaserawsponsor \
    left join dse203finalproject.StateReps on xmlbaserawsponsor.LastName = StateReps.LastName \
    and xmlbaserawsponsor.State = StateReps.Statelabel where xmlbaserawsponsor.State is NOT NULL and \
    StateReps.RepresentativeID is NOT NULL'
#cursor.execute(querysponin2)
querycspon2='CREATE TABLE dse203finalproject.CosponsorsbyID(BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, RepresentativeID INT references \
    dse203finalproject.StateReps (RepresentativeID) NOT NULL)'    
#cursor.execute(querycspon2)
querycsponin2='INSERT INTO dse203finalproject.CosponsorsbyID (BillID, RepresentativeID) \
Select xmlbaserawcosponsor.BillID as "BillID", StateReps.RepresentativeID as "RepresentativeID" from dse203finalproject.xmlbaserawcosponsor \
    left join dse203finalproject.StateReps on xmlbaserawcosponsor.LastName = StateReps.LastName \
    where xmlbaserawcosponsor.State is NULL and StateReps.RepresentativeID is NOT NULL \
    UNION \
    Select xmlbaserawcosponsor.BillID as "BillID", StateReps.RepresentativeID as "RepresentativeID" from dse203finalproject.xmlbaserawcosponsor \
    left join dse203finalproject.StateReps on xmlbaserawcosponsor.LastName = StateReps.LastName \
    and xmlbaserawcosponsor.State = StateReps.Statelabel where xmlbaserawcosponsor.State is NOT NULL and \
    StateReps.RepresentativeID is NOT NULL'
#cursor.execute(querycsponin2)